![paper screen](img/paper_screen.png)

# Plan

* Markov decision process terminology and Q-learning
* Problem-specific notation
* Q-knn

# Market making as a refinforcement learning/stochastic control problem

The paper at hand approaches the problem of market making as an optimal control / reinforcement learning problem.

It seeks to find an optimal quality function to describe current state of market and an optimal policy to take actions.

# Markov Decision Process

Optimal control theory works with a probabilistic environment, the state of which is fully described by a state $Z_t$ (one might pay attention to the time $t$ or ignore it, dropping the sub-index and considering just the state $Z$, irrespective of time).

The process is Markovian because transition from $Z_t$ to $Z_{t+1}$ is described by a transition probability matrix/kernel $Q$ (don't confuse it with Q-function, used later, though).

The agent can take an action $\alpha_t$ at each time step $t$, receiving a reward $r_t$ and transitioning the system into state $Z_{t+1}$. Generally the transition is probabilistic and actions can be probabilistic as well.

The goal is to find an optimal strategy/policy $A$/$\pi$ from a certain space of valid strategies $\mathbb{A}$, which optimizes the total reward $\sum \limits_t r_t$ an agent gets over its lifetime.

# Actions and rewards in MDP

For each state of the system $z_t$ we can select an action from a space $A_z$.

By choosing an action $\alpha_z$, we obtain a reward $r$. 

Hence, there is a reward function $R: Z_t \times A_z \to \mathbb{R}$, which defines the rewards, associated with each action at each state.

![q_learning](img/q_learning.png)

# Value function, reward function, quality function (a.k.a. Q-function, action-value function)

Value function is the estimate of the current position at the moment of time $t$ and state $z$. It represents the expectation of maximum reward you can attain, if you play your cards optimally.

$V(T, z) = \sup_{\alpha \in \mathbb{A}} \mathbb{E} \lbrack \int \limits_{0}^{T} R_s(z_s) ds \rbrack$, where $R_s: A_z \times Z_s \to \mathbb{R}$ is the reward that can be attained at the moment $s$, when the system is in a state $z_s$.

Let us decompose the reward function $R_s$ into instantaneous reward $f(\alpha_s, Z_s)$ at step $s$ and terminal reward $g(Z_T)$: $R_s = f(\alpha_s, z_s) ds + g(z_T)$.

$V(t, z) = \sup_{\alpha \in \mathbb{A}} \mathbb{E} \lbrack \int \limits_{t}^{T} f(\alpha_s, Z_s) ds + g(Z_T) \rbrack$

Let us explicitly reflect the fact that Value function depends on the action $\alpha_s$ we take at each moment $s$ according to our policy/strategy $A_s$.

Introduce an action-value function $Q_t(z, \alpha)$, relfecting the terminal return we get upon entering the state.

Optimal strategy $A_s \in \mathbb{A}$ produces the optimal action-value function $Q^*_t(z, \alpha)$, such that $Q^*_t(z, \alpha) = V(t, z)$:

$Q^*_t(z_t, \alpha_t) = f(\alpha, z) + \gamma \mathbb{E}_{p(z_{t+1} | z_t, \alpha_t)} \lbrack \max_{\alpha_{t+1}} Q^*_{t_0}(z_{t+1}, \alpha_{t+1}) | z_t, \alpha_t \rbrack$ , where $\gamma$ is a discount factor for delayed reward.

This recursion is Bellman's equation.

# TD-learning, TD(0)

We need to come up with a way to estimate the value function $V(S)$ and/or quality function $Q(S,A)$.

The general approach is iterative.

Recall the general first order optimization: $V^{new}(s) = V^{old}(s) + lr \cdot \Delta V(s)$, where $lr$ is step size/learning rate and $\Delta V$ is improvement of estimate over one step. However, upon meaningful steps your state also changes, and we need to take this into account. Substitute Bellman's equation as $\Delta V$:

$V^{new}(s_t) = V^{old}(s_t) + lr \cdot \overbrace{(\underbrace{r_t + \gamma V^{old}(s_{t+1})}_{\text{TD estimate of V}} - V^{old}(s_t))}^{\text{TD error}}$

This is an example of Temporal Difference learning (TD-learning). Specifically, this is 1-step deep TD-learning, TD(0).

# TD(1)

We can go deeper:

$V^{new}(s_t) = V^{old}(s_t) + lr \cdot \overbrace{(\underbrace{r_t + \gamma r_{t+1} + \gamma^2 V^{old}(s_{t+2})}_{\text{TD(1) estimate of V}} - V^{old}(s_t))}^{\text{TD(1) error}}$

One might pick arbitrary N for TD(N).

# Q-learning

When applied to quality function instead of value function, we get:

$Q^{new}(s_t, \alpha_t) = V^{old}(s_t, \alpha_t) + lr \cdot \overbrace{(\underbrace{r_t + \gamma \max\limits_{\alpha} Q^{old}(s_{t+1}, \alpha)}_{\text{TD estimate}} - Q^{old}(s_t, \alpha_t))}^{\text{TD error}}$

Importantly, $\alpha_t$ here does **not** necesserily need to be optimal. One might observe other person take actions, get a change in quality function and learn from their mistakes.

This case is called **off-policy learning** in contrast to on-policy learning, when next actions are generated according to our policy in algorithms, such as SARSA.

# Q-learning

![q-learning 2](img/q_learning2.png)

Here epsilon-greedy strategy is a strategy of choice of actions, based on a balance of exploration and exploitation, which exploits the optimal action, found so far in most cases and explores new options with probability $\epsilon$.

# State space: current problem

$Z_t = X_t, Y_t, a_t, b_t, na_t, nb_t, pa_t, pb_t, ra_t,rb_t$, where:

* $X_t$ - cash held by marketmaker
* $Y_t$ - inventory of marketmaker
* $a_t = (a_1, ..., a_K)$ - bbo ask levels 1..K
* $b_t = (b_1, ..., b_K)$ - bbo bid levels 1..K
* $na_t = (na_1, ..., nb_K)$ - ranks of marketmaker ask orders in the respective level's order queue
* $nb_t = (nb_1, ..., nb_K)$ - ranks of marketmaker bid orders in the respective level's order queue
* $pa_t$ - bbo ask price
* $pb_t$ - bbo bid price
* $ra_t = (ra_1, ..., ra_K)$ - market maker ask order positions
* $rb_t = (rb_1, ..., rb_K)$ - market maker bid order positions

![orders](img/orders.png)

# Current problem as a Markov decision process

## Notation

* $Z_T$ - state-time vector space, $[0, T] × E$ - state space;
* $A_z \in \mathbb{A}$ - market maker control, we are seeking, consists of actions $\alpha_t$; $\mathbb{A}$ is the set of all admissible strategies;
* $\lambda$ - intensity of the jump;
* $Q$- transitions kernel;
* $r$ - reward;

The process is Markov, because current state of the system is completely described by the state vector $z_t$ and depends only on the previous state and action taken.

Our goal is to maximize the value function of Markov decision process.

# Infinite state space 

In general state space might be too large to traverse it completely. E.g. in chess the number of states possible could be $10^80$.

What approaches can we take to overcome this problem?

* Reparametrization: e.g. DQN - instead of traversing the state of spaces, let us traverse a relatively low-dimensional state of Neural Network parameters
* MCMC approach
* non-parametric kNN + quantization approach

# k-Nearest Neighbours regression

A simple non-parametric method for regression/classification:

![knn](img/knn.png)

# Algorithm

![algorithm](img/algorithm.png)

# References:
* https://www.youtube.com/watch?v=BvQVovMefsM - kNN
* https://www.cs.toronto.edu/~rgrosse/courses/csc421_2019/slides/lec21.pdf - UToronto course on Q-learning
* https://www.youtube.com/watch?v=0iqz4tcKN58 - Steve Brunson on TD-learning and Q-learning